In [1]:
import os
import config

import pandas as pd
import numpy as np
from scipy.stats import beta

from flask import Flask, jsonify, render_template

import mysql.connector

from datetime import datetime, date, time, timedelta
from time import sleep

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import requests

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

In [2]:
# Fetch the service account key JSON file contents
cred = credentials.Certificate(config.firebase_file_path)
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': config.databaseURL
})


In [3]:




import os, time
time.strftime('%X %x %Z')

os.environ['TZ'] = 'Europe/London'
time.tzset()

time.strftime('%X %x %Z')

from pytz import timezone

now = datetime.now()
now = timezone('US/Central').localize(now)

nowMonth=now.month
nowYear=now.year
nowMonth

today=datetime.today()
first = today.replace(day=1)
lastMonth = first - timedelta(days=1)
lastMonthYear=lastMonth.year 
lastMonthMonth=lastMonth.month


orders_db = "orders.json"
r = requests.get(config.databaseURL + orders_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    orders_df = pd.DataFrame.from_dict(data, orient='columns')
orders_df = orders_df.astype({"orderID": int})

orders_df=orders_df[['orderID','orderDate']]

orderShipped_db = "order_shipped.json"
r = requests.get(config.databaseURL + orderShipped_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    orders_shipped_df= pd.DataFrame.from_dict(data, orient='columns')
orders_shipped_df = orders_shipped_df.astype({"orderID": int})

order_2=pd.merge(orders_df,orders_shipped_df,on='orderID',how='outer')

order_2

,orderID,orderDate,dateShipped
0,1441612324,2019-04-19,04/28/19
1,1461258703,2019-05-15,05/28/19
2,1465581068,2019-06-17,06/20/19
3,1467289834,2019-06-21,07/05/19
4,1471406642,2019-07-02,07/05/19
5,1471997258,2019-07-04,07/12/19
6,1479218984,2019-07-23,07/29/19
7,1482734683,2019-07-09,07/12/19
8,1487089393,2019-07-20,07/21/19
9,1480128610,2019-07-26,07/27/19


In [4]:
order_2['orderDate'] =  pd.to_datetime(order_2['orderDate'])
order_2['dateShipped'] =  pd.to_datetime(order_2['dateShipped'], format='%m/%d/%y')
order_2["month"] = order_2['orderDate'].map(lambda x: x.month)
order_2["year"] = order_2['orderDate'].map(lambda x: x.year)

In [5]:
order_2

,orderID,orderDate,dateShipped,month,year
0,1441612324,2019-04-19,2019-04-28,4,2019
1,1461258703,2019-05-15,2019-05-28,5,2019
2,1465581068,2019-06-17,2019-06-20,6,2019
3,1467289834,2019-06-21,2019-07-05,6,2019
4,1471406642,2019-07-02,2019-07-05,7,2019
5,1471997258,2019-07-04,2019-07-12,7,2019
6,1479218984,2019-07-23,2019-07-29,7,2019
7,1482734683,2019-07-09,2019-07-12,7,2019
8,1487089393,2019-07-20,2019-07-21,7,2019
9,1480128610,2019-07-26,2019-07-27,7,2019


In [6]:
prep_time = order_2[pd.notnull(order_2['dateShipped'])] 

In [7]:
prep_time

,orderID,orderDate,dateShipped,month,year
0,1441612324,2019-04-19,2019-04-28,4,2019
1,1461258703,2019-05-15,2019-05-28,5,2019
2,1465581068,2019-06-17,2019-06-20,6,2019
3,1467289834,2019-06-21,2019-07-05,6,2019
4,1471406642,2019-07-02,2019-07-05,7,2019
5,1471997258,2019-07-04,2019-07-12,7,2019
6,1479218984,2019-07-23,2019-07-29,7,2019
7,1482734683,2019-07-09,2019-07-12,7,2019
8,1487089393,2019-07-20,2019-07-21,7,2019
9,1480128610,2019-07-26,2019-07-27,7,2019


In [8]:
prep_time["prepTime"] = prep_time["dateShipped"].sub(prep_time["orderDate"], axis=0)
prep_time['prepTime'] = prep_time['prepTime'] / np.timedelta64(1, 'D')

In [9]:
prep_time

,orderID,orderDate,dateShipped,month,year,prepTime
0,1441612324,2019-04-19,2019-04-28,4,2019,9.0
1,1461258703,2019-05-15,2019-05-28,5,2019,13.0
2,1465581068,2019-06-17,2019-06-20,6,2019,3.0
3,1467289834,2019-06-21,2019-07-05,6,2019,14.0
4,1471406642,2019-07-02,2019-07-05,7,2019,3.0
5,1471997258,2019-07-04,2019-07-12,7,2019,8.0
6,1479218984,2019-07-23,2019-07-29,7,2019,6.0
7,1482734683,2019-07-09,2019-07-12,7,2019,3.0
8,1487089393,2019-07-20,2019-07-21,7,2019,1.0
9,1480128610,2019-07-26,2019-07-27,7,2019,1.0


In [10]:
prepTimePrev = prep_time.loc[prep_time['month'] == lastMonthMonth,:]

In [11]:
prepTimePrev

,orderID,orderDate,dateShipped,month,year,prepTime
4,1471406642,2019-07-02,2019-07-05,7,2019,3.0
5,1471997258,2019-07-04,2019-07-12,7,2019,8.0
6,1479218984,2019-07-23,2019-07-29,7,2019,6.0
7,1482734683,2019-07-09,2019-07-12,7,2019,3.0
8,1487089393,2019-07-20,2019-07-21,7,2019,1.0
9,1480128610,2019-07-26,2019-07-27,7,2019,1.0
10,1480472568,2019-07-27,2019-08-16,7,2019,20.0
15,1481079872,2019-07-29,2019-07-30,7,2019,1.0
16,1481051598,2019-07-28,2019-07-30,7,2019,2.0


In [12]:
prepTimeAvg_prev=prepTimePrev['prepTime'].mean()
prepTimeAvg_prev

5.0

In [13]:
prepTimeCur = prep_time.loc[prep_time['month'] == nowMonth,:]
prepTimeCur

,orderID,orderDate,dateShipped,month,year,prepTime
11,1484941268,2019-08-07,2019-08-12,8,2019,5.0
12,1493211993,2019-08-05,2019-08-19,8,2019,14.0
13,1486172004,2019-08-10,2019-08-19,8,2019,9.0
14,1482480540,2019-08-01,2019-08-05,8,2019,4.0
17,1489256300,2019-08-18,2019-08-19,8,2019,1.0
18,1498480151,2019-08-18,2019-08-19,8,2019,1.0


In [14]:
prepTimeAvg_cur=prepTimeCur['prepTime'].mean()


In [15]:
prepTimeDiff=(prepTimeAvg_cur/prepTimeAvg_prev)*100
prepTimeDiff=str(round(prepTimeDiff,1))
prepTimeDiff=f"last month {prepTimeAvg_prev} days"
prepTimeDiff

'last month 5.0 days'

In [16]:
prepTime=[]


In [17]:
prepTime.append(prepTimeAvg_cur)
prepTime.append(prepTimeDiff)

In [18]:
order_count3 = order_2.loc[order_2['year'] == nowYear,:]
order_count_now = order_count3.loc[order_count3['month'] == nowMonth,:]


orderCount=[]
count_cur=order_count_now['orderID'].count()

In [19]:
count_curr=int(count_cur)

In [20]:
count_curr

6

In [21]:
order_count4 = order_2.loc[order_2['year'] == lastMonthYear,:]
order_count_lm = order_count4.loc[order_count4['month'] == lastMonthMonth,:]


In [22]:
order_count_lm

,orderID,orderDate,dateShipped,month,year
4,1471406642,2019-07-02,2019-07-05,7,2019
5,1471997258,2019-07-04,2019-07-12,7,2019
6,1479218984,2019-07-23,2019-07-29,7,2019
7,1482734683,2019-07-09,2019-07-12,7,2019
8,1487089393,2019-07-20,2019-07-21,7,2019
9,1480128610,2019-07-26,2019-07-27,7,2019
10,1480472568,2019-07-27,2019-08-16,7,2019
15,1481079872,2019-07-29,2019-07-30,7,2019
16,1481051598,2019-07-28,2019-07-30,7,2019


In [23]:


count_prev=order_count_lm['orderID'].count()

In [24]:
count_prev = int(count_prev)

In [25]:
count_prev

9

In [26]:
count_diff=(count_cur/count_prev)*100
# count_diff='{0:+d}'.format(count_diff)
count_diff

66.66666666666666

In [27]:
count_diff=str(round(count_diff,1))
count_diff=f"{count_diff}%"
count_diff

'66.7%'

In [28]:
orderCount.append(count_cur)
orderCount.append(count_diff)

In [30]:
orderNet = "order_net.json"
r = requests.get(config.databaseURL + orderNet)
r = r.json()

if r:
    data = [r[i] for i in r]
    orderNet_df2= pd.DataFrame.from_dict(data, orient='columns')

orderNet_df2["date"] = pd.to_datetime(orderNet_df2["date"])
orderNet_df2 = orderNet_df2.astype({"orderNet": float})

orderNet_df2

,date,orderNet
0,2019-07-20,180.26
1,2019-07-21,180.06
2,2019-07-22,180.06
3,2019-07-23,180.06
4,2019-07-24,214.40
5,2019-07-25,214.40
6,2019-07-26,214.40
7,2019-07-27,281.15
8,2019-07-28,911.52
9,2019-07-29,933.60


In [31]:
orderNet_df2["month"] = orderNet_df2['date'].map(lambda x: x.month)
orderNet_df2["year"] = orderNet_df2['date'].map(lambda x: x.year)


In [32]:
orderNet_df2 = orderNet_df2.groupby(['year','month'], as_index=False)['orderNet'].max()            


In [33]:
orderNet_df2

,year,month,orderNet
0,2019,7,933.60
1,2019,8,506.22


In [34]:
orderNet_df3 = orderNet_df2.loc[orderNet_df2['month'] == lastMonthMonth,:]

In [35]:
orderNet_df3

,year,month,orderNet
0,2019,7,933.6


In [36]:
orderNet_df4 = orderNet_df2.loc[orderNet_df2['month'] == nowMonth,:]
orderNet_df4

,year,month,orderNet
1,2019,8,506.22


In [37]:
net_prev=orderNet_df3.iloc[0]['orderNet']
net_cur=orderNet_df4.iloc[0]['orderNet']

In [38]:
print(net_prev)
print(net_cur)

933.6
506.22


In [39]:
orderNetdiff=(net_cur/net_prev)*100

order_net_diff=str(round(orderNetdiff,1))

order_net_diff=f"{order_net_diff}%"

In [40]:
orderNetAmt=orderNet_df4.iloc[0]['orderNet']
orderNetAmt=str(orderNetAmt)


In [41]:
orderNet=[]
orderNet.append(orderNetAmt)
orderNet.append(order_net_diff)

order_stats_df=pd.DataFrame({'avgOrderTime':prepTime,
                    'orderCount':orderCount,
                    'orderNet':orderNet})

order_stats=[]

for index, item in order_stats_df.iterrows():
    orderStatsDict={}
    orderStatsDict['avgOrderTime']=item[0]
    orderStatsDict['orderCount']=item[1]
    orderStatsDict['orderRevenue']=item[2]

    order_stats.append(orderStatsDict)

order_stats

[{'avgOrderTime': 5.666666666666667,
  'orderCount': 6,
  'orderRevenue': '506.22'},
 {'avgOrderTime': 'last month 5.0 days',
  'orderCount': '66.7%',
  'orderRevenue': '54.2%'}]